In [ ]:
import torch
from torch.utils.data import Dataset
import os 
from path import Path

In [2]:
def read_off(file):
    if 'OFF' != file.readline().strip():
        raise('Not a valid OFF header')
    n_verts, n_faces, __ = tuple([int(s) for s in file.readline().strip().split(' ')])
    verts = [[float(s) for s in file.readline().strip().split(' ')] for i_vert in range(n_verts)]
    faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for i_face in range(n_faces)]
    return verts, faces

In [8]:
class PointCloudData(Dataset):
    def __init__(self, root_dir, valid=False, folder="train"):
        self.root_dir = root_dir
        folders = [dir for dir in sorted(os.listdir(root_dir)) if os.path.isdir(root_dir/dir)]
        self.classes = {folder: i for i, folder in enumerate(folders)}
        self.valid = valid
        self.files = []
        for category in self.classes.keys():
            new_dir = root_dir/Path(category)/folder
            for file in os.listdir(new_dir):
                if file.endswith('.off'):
                    sample = {}
                    sample['pcd_path'] = new_dir/file
                    sample['category'] = category
                    self.files.append(sample)

    def __len__(self):
        return len(self.files)
    

    def __getitem__(self, idx):
        pcd_path = self.files[idx]['pcd_path']
        category = self.files[idx]['category']
        with open(pcd_path, 'r') as f:
            pointcloud = self.__preproc__(f)
        return {'pointcloud': pointcloud, 
                'category': self.classes[category]}

In [9]:
path = Path("ModelNet10")
test_dataset = PointCloudData(path, True)

In [1]:
import open3d as o3d
import numpy as np
for i in range(99,100, 1):
    filename = "/home/akshay/Deep_Closest_Point/dcpv-1_org/sample_input_" + str(i) + ".npy"
    np_array = np.load(filename)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np_array)
    o3d.visualization.draw_geometries([pcd])

    filename = "/home/akshay/Deep_Closest_Point/dcpv-1_org/sample_output" + str(i) + ".npy"
    np_array = np.load(filename)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np_array)
    o3d.visualization.draw_geometries([pcd])


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
from idgcn import ConvBNList, find_neighbors, get_graph_feature
import torch
from pointnet2_ops.pointnet2_utils import grouping_operation
import time

In [6]:
x = torch.randn((64, 3, 1024)).cuda()
y = find_neighbors(x)
y = y.type(torch.int32).contiguous().cuda()
start_t = time.time()
z = get_graph_feature(x, y)
print(time.time()-start_t)
start_t = time.time()
z2 = grouping_operation(x.cuda(), y.cuda())
print(time.time()-start_t)
z.shape, z2.shape

0.0004763603210449219
0.00010132789611816406


(torch.Size([64, 6, 1024, 20]), torch.Size([64, 3, 1024, 20]))

In [4]:
agg_fn = lambda x: x
layer1 = ConvBNList([32, 64], agg_fn, 128)
print(layer1)

Conv_BN_List(
  (layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [9]:
%reload_ext autoreload
import torch
from idgcn import DenseGCN
layer = DenseGCN(32, 128)
print(layer)
x = torch.randn((8, 32, 1024, 20))
y = layer(x)
y.shape

DenseGCN(
  (conv_bn): ConvBNList(
    (layers): ModuleList(
      (0): Sequential(
        (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): LeakyReLU(negative_slope=0.2)
  )
)


torch.Size([8, 128, 1024, 20])

In [ ]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
import torch 
from idgcn import InceptionDenseGCN
layer = InceptionDenseGCN(128, 256).cuda()
x = torch.randn((3, 128, 1024)).cuda()
print(layer)
y = layer(x)
y.shape

In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
import torch
import time 
from idgcn import FeatureExtractor
layer = FeatureExtractor(features_dim=[3, 128, 256, 256]).cuda()
x = torch.randn((8, 3, 1024)).cuda()
start_t = time.time()
y = layer(x)
print(time.time() - start_t)
y.shape


0.35939550399780273


torch.Size([8, 512, 1024])

In [39]:
from model import DGCNN
import time
model = DGCNN().cuda()
x = torch.randn((8, 3, 1024)).cuda()
start_t = time.time()
y = model(x)
print(time.time() - start_t)
x = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(x)

0.0014646053314208984
309632
